# Fusion Retrieval in Document Search

## Overview

This code implements a Fusion Retrieval system that combines vector-based similarity search with keyword-based BM25 retrieval. The approach aims to leverage the strengths of both methods to improve the overall quality and relevance of document retrieval.

## Motivation

Traditional retrieval methods often rely on either semantic understanding (vector-based) or keyword matching (BM25). Each approach has its strengths and weaknesses. Fusion retrieval aims to combine these methods to create a more robust and accurate retrieval system that can handle a wider range of queries effectively.

## Key Components

1. PDF processing and text chunking
2. Vector store creation using FAISS and OpenAI embeddings
3. BM25 index creation for keyword-based retrieval
4. Custom fusion retrieval function that combines both methods

## Method Details

### Document Preprocessing

1. The PDF is loaded and split into chunks using RecursiveCharacterTextSplitter.
2. Chunks are cleaned by replacing 't' with spaces (likely addressing a specific formatting issue).

### Vector Store Creation

1. OpenAI embeddings are used to create vector representations of the text chunks.
2. A FAISS vector store is created from these embeddings for efficient similarity search.

### BM25 Index Creation

1. A BM25 index is created from the same text chunks used for the vector store.
2. This allows for keyword-based retrieval alongside the vector-based method.

### Fusion Retrieval Function

The `fusion_retrieval` function is the core of this implementation:

1. It takes a query and performs both vector-based and BM25-based retrieval.
2. Scores from both methods are normalized to a common scale.
3. A weighted combination of these scores is computed (controlled by the `alpha` parameter).
4. Documents are ranked based on the combined scores, and the top-k results are returned.

## Benefits of this Approach

1. Improved Retrieval Quality: By combining semantic and keyword-based search, the system can capture both conceptual similarity and exact keyword matches.
2. Flexibility: The `alpha` parameter allows for adjusting the balance between vector and keyword search based on specific use cases or query types.
3. Robustness: The combined approach can handle a wider range of queries effectively, mitigating weaknesses of individual methods.
4. Customizability: The system can be easily adapted to use different vector stores or keyword-based retrieval methods.

## Conclusion

Fusion retrieval represents a powerful approach to document search that combines the strengths of semantic understanding and keyword matching. By leveraging both vector-based and BM25 retrieval methods, it offers a more comprehensive and flexible solution for information retrieval tasks. This approach has potential applications in various fields where both conceptual similarity and keyword relevance are important, such as academic research, legal document search, or general-purpose search engines.

<div style="text-align: center;">

<img src="../images/fusion_retrieval.svg" alt="Fusion Retrieval" style="width:100%; height:auto;">
</div>

# Package Installation and Imports

The cell below installs all necessary packages required to run this notebook.


In [1]:
import sys
from pathlib import Path

# 1. Define the directory *containing* the all_rag_techniques package
# Get the directory of the current notebook/script (__file__ might not work in some notebooks)
# Assuming the notebook is inside all_rag_techniques/
current_dir = Path.cwd() 

# The directory containing 'all_rag_techniques' is the parent directory
project_root = current_dir.parent 

# 2. Add this root to the system path
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))
    print(f"Added project root to path: {project_root}")
else:
    print("Project root already in path.")

# 3. Now the import should work
try:
    from all_rag_techniques import setup_environment, check_keys
    print("✅ Package imported successfully!")
    setup_environment()
    check_keys()
except Exception as e:
    print(f"❌ Final import failed: {e}")

Added project root to path: /Users/ruhwang/Desktop/AI/my_projects/context-engineering/advanced-rag
✅ Package imported successfully!
LANGCHAIN_API_KEY not set (empty in .env file)
Environment setup complete!
=== API Keys from config.py ===
  GROQ_API_KEY: Loaded
  COHERE_API_KEY: Loaded
  OPENAI_API_KEY: Loaded
  LANGCHAIN_API_KEY: Missing

=== Environment Variables ===
  os.environ['GROQ_API_KEY']: Set
  os.environ['COHERE_API_KEY']: Set

All essential keys loaded!


In [ ]:
# Install required packages
!pip install rank-bm25

In [ ]:

"""
# Clone the repository to access helper functions and evaluation modules
!git clone https://github.com/NirDiamant/RAG_TECHNIQUES.git
import sys
sys.path.append('RAG_TECHNIQUES')
# If you need to run with the latest data
# !cp -r RAG_TECHNIQUES/data .

# Load environment variables from a .env file
load_dotenv()

# Set the OpenAI API key environment variable
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

# Download required data files
import os
os.makedirs('data', exist_ok=True)

# Download the PDF document used in this notebook
!wget -O data/Understanding_Climate_Change.pdf https://raw.githubusercontent.com/NirDiamant/RAG_TECHNIQUES/main/data/Understanding_Climate_Change.pdf
!wget -O data/Understanding_Climate_Change.pdf https://raw.githubusercontent.com/NirDiamant/RAG_TECHNIQUES/main/data/Understanding_Climate_Change.pdf

"""

In [3]:
import os
import sys
from dotenv import load_dotenv
from langchain.docstore.document import Document

from typing import List
from rank_bm25 import BM25Okapi
import numpy as np


# Original path append replaced for Colab compatibility
from helper_functions import *
from evaluation.evalute_rag import *

### Define document path

In [4]:
path = "/Users/ruhwang/Desktop/AI/my_projects/context-engineering/advanced-rag/data/Understanding_Climate_Change.pdf"

### Encode the pdf to vector store and return split document from the step before to create BM25 instance

In [5]:
def encode_pdf_and_get_split_documents(path, chunk_size=1000, chunk_overlap=200):
    """
    Encodes a PDF book into a vector store using OpenAI embeddings.

    Args:
        path: The path to the PDF file.
        chunk_size: The desired size of each text chunk.
        chunk_overlap: The amount of overlap between consecutive chunks.

    Returns:
        A FAISS vector store containing the encoded book content.
    """

    # Load PDF documents
    loader = PyPDFLoader(path)
    documents = loader.load()

    # Split documents into chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, chunk_overlap=chunk_overlap, length_function=len
    )
    texts = text_splitter.split_documents(documents)
    cleaned_texts = replace_t_with_space(texts)

    # Create embeddings and vector store
    embeddings = OpenAIEmbeddings(model="text-embedding-3-small", dimensions=512)
    vectorstore = FAISS.from_documents(cleaned_texts, embeddings)

    return vectorstore, cleaned_texts

### Create vectorstore and get the chunked documents

In [6]:
vectorstore, cleaned_texts = encode_pdf_and_get_split_documents(path)

### Create a bm25 index for retrieving documents by keywords

In [7]:
def create_bm25_index(documents: List[Document]) -> BM25Okapi:
    """
    Create a BM25 index from the given documents.

    BM25 (Best Matching 25) is a ranking function used in information retrieval.
    It's based on the probabilistic retrieval framework and is an improvement over TF-IDF.

    Args:
    documents (List[Document]): List of documents to index.

    Returns:
    BM25Okapi: An index that can be used for BM25 scoring.
    """
    # Tokenize each document by splitting on whitespace
    # This is a simple approach and could be improved with more sophisticated tokenization
    tokenized_docs = [doc.page_content.split() for doc in documents]
    return BM25Okapi(tokenized_docs)

In [8]:
bm25 = create_bm25_index(cleaned_texts) # Create BM25 index from the cleaned texts (chunks)

In [9]:
bm25

BM25 is an improvement over TF-IDF, based on probabilistic retrieval

### Define a function that retrieves both semantically and by keyword, normalizes the scores and gets the top k documents

In [ ]:
def fusion_retrieval(vectorstore, bm25, query: str, k: int = 5, alpha: float = 0.5) -> List[Document]:
    """
    Perform fusion retrieval combining keyword-based (BM25) and vector-based search.

    Args:
    vectorstore (VectorStore): The vectorstore containing the documents.
    bm25 (BM25Okapi): Pre-computed BM25 index.
    query (str): The query string.
    k (int): The number of documents to retrieve.
    alpha (float): The weight for vector search scores (1-alpha will be the weight for BM25 scores).

    Returns:
    List[Document]: The top k documents based on the combined scores.
    """
    
    epsilon = 1e-8

    # Step 1: Get all documents from the vectorstore
    all_docs = vectorstore.similarity_search("", k=vectorstore.index.ntotal)

    # Step 2: Perform BM25 search
    bm25_scores = bm25.get_scores(query.split()) # list of scores for all documents 

    # Step 3: Perform vector search
    vector_results = vectorstore.similarity_search_with_score(query, k=len(all_docs)) 
    #list of (Document, score) tuples of length equal to all_docs
    
    # Step 4: Normalize scores
    vector_scores = np.array([score for _, score in vector_results]) # extract scores in a numpy array
    vector_scores = 1 - (vector_scores - np.min(vector_scores)) / (np.max(vector_scores) - np.min(vector_scores) + epsilon)
    # normalized BM25 scores with MinMax normalization
    bm25_scores = (bm25_scores - np.min(bm25_scores)) / (np.max(bm25_scores) -  np.min(bm25_scores) + epsilon)

    # Step 5: Combine scores
    combined_scores = alpha * vector_scores + (1 - alpha) * bm25_scores  

    # Step 6: Rank documents
    sorted_indices = np.argsort(combined_scores)[::-1] 
    # in descending order, np.argsort gives ascending order, and [::-1] reverses it
    
    # Step 7: Return top k documents
    return [all_docs[i] for i in sorted_indices[:k]]

Both vector_scores and bm25_scores are first normalized to the range [0, 1] using min-max scaling (higher score = more relevant).

{combined_scores} is a linear weighted combination (or convex combination) of the normalized vector search scores and BM25 scores. It fuses the two retrieval methods into a single score per document, balancing their contributions based on the alpha parameter. The goal is to leverage the strengths of both: vector search (semantic similarity) and BM25 (keyword matching)

1. For vectors: 1 - (score - min) / (max - min + epsilon) (inverted because vector distances are often smaller = better, but here it's adapted to make higher = better).

2. For BM25: (score - min) / (max - min + epsilon) (BM25 scores are naturally higher = better).
The formula alpha * vector_scores + (1 - alpha) * bm25_scores is a weighted average:
It ensures the weights sum to 1 (alpha + (1 - alpha) = 1), so the combined scores stay in [0, 1].

3. For each document i: combined_scores[i] = alpha * vector_scores[i] + (1 - alpha) * bm25_scores[i].
Example: If alpha = 0.7, a document gets 70% weight from its vector score and 30% from its BM25 score.

This promotes documents that perform well in both methods (reciprocal rank fusion-like, but score-based). After combination, sorting (as explained above) ranks them for the final top-k output.

### Use Case example

In [15]:
# Query
query = "What are the impacts of climate change on the environment?"

# Perform fusion retrieval
top_docs = fusion_retrieval(vectorstore, bm25, query, k=5, alpha=0.5)
docs_content = [doc.page_content for doc in top_docs]
show_context(docs_content)

Context 1:
challenges. This includes collaboration between scientists, policymakers, businesses, and 
communities. Interdisciplinary research and solutions are more holistic and effective. 
Citizen Science 
Citizen science involves engaging the public in scientific research and data collection. This 
empowers individuals to contribute to climate knowledge and action. Citizen science projects 
can enhance data accuracy, raise awareness, and foster community engagement. 
Hope and Inspiration 
Positive Narratives


Context 2:
trust and achieving policy objectives. 
International Climate Negotiations 
Conference of the Parties (COP) 
The Conference of the Parties (COP) is the supreme decision-making body of the United 
Nations Framework Convention on Climate Change (UNFCCC). COP meetings bring 
together representatives from all signatory countries to negotiate and review progress on 
climate agreements. These meetings play a crucial role in advancing global climate action. 
Climate Finance

[(Document(id='40890252-3bfd-4661-a872-a4e83d651d50', metadata={'producer': 'Microsoft® Word 2021', 'creator': 'Microsoft® Word 2021', 'creationdate': '2024-07-13T20:17:34+03:00', 'author': 'Nir', 'moddate': '2024-07-13T20:17:34+03:00', 'source': '/Users/ruhwang/Desktop/AI/my_projects/context-engineering/advanced-rag/data/Understanding_Climate_Change.pdf', 'total_pages': 33, 'page': 12, 'page_label': '13'}, page_content='Climate change is altering terrestrial ecosystems by shifting habitat ranges, changing species \ndistributions, and impacting ecosystem functions. Forests, grasslands, and deserts are \nexperiencing shifts in plant and animal species composition. These changes can lead to a loss \nof biodiversity and disrupt ecological balance. \nMarine Ecosystems \nMarine ecosystems are highly vulnerable to climate change. Rising sea temperatures, ocean \nacidification, and changing currents affect marine biodiversity, from coral reefs to deep-sea \nhabitats. Species migration and changes in reproductive cycles can disrupt marine food webs \nand fisheries.'), 
...
]

![](https://europe-west1-rag-techniques-views-tracker.cloudfunctions.net/rag-techniques-tracker?notebook=all-rag-techniques--fusion-retrieval)